In [5]:
def aux(funcs, acc, handle_result, debug=False, depth=0):
    # Helper to print debug messages with indentation
    def debug_log(message):
        if debug:
            print(f"{'  ' * depth}{message}")
    
    debug_log(f"aux called with acc={acc}, funcs={[f.__name__ if hasattr(f, '__name__') else '<lambda>' for f in funcs]}")

    if not funcs:
        result = handle_result(acc)
        debug_log(f"No more funcs. Final result={result}")
        return result
    
    head, *tail = funcs
    debug_log(f"Processing head={head.__name__ if hasattr(head, '__name__') else '<lambda>'} with acc={acc}")

    def skip_continuation(skip_result):
        debug_log(f"  skip_continuation called with skip_result={skip_result}")

        def apply_continuation(apply_result):
            debug_log(f"  apply_continuation called with apply_result={apply_result}")
            combined_result = max(skip_result, apply_result)
            debug_log(f"combined_result={combined_result}")
            return handle_result(combined_result)

        return aux(tail, head(acc), apply_continuation, debug=debug, depth=depth + 1)

    result = aux(tail, acc, skip_continuation, debug=debug, depth=depth + 1)
    debug_log(f"Returning result={result}")
    return result



In [6]:
def test_aux():
    # Identity function to use as a simple `handle_result`
    identity = lambda x: x

    # Test case 1: No functions, just return initial value
    funcs = []
    init = 5
    assert aux(funcs, init, identity) == 5, "Test case 1 failed"

    # Test case 2: Single function in the list
    funcs = [lambda x: x + 2]
    init = 5
    assert aux(funcs, init, identity) == 7, "Test case 2 failed"

    # Test case 3: Multiple functions
    # Define named functions
    def add_four(x):
        return x + 4

    def multiply_by_two(x):
        return x * 2
    funcs = [
        add_four,  # Add 1
        multiply_by_two,  # Multiply by 2
    ]
    init = 1
    assert aux(funcs, init, identity, debug=True) == 10, "Test case 3 failed"
    # Explanation:
    # Skip: 5 -> 5 -> 5
    # Apply: 5 -> 6 -> 12 -> 9
    # Max at each step is 10 (skip + apply combined logic)

    # Test case 4: Using a different handle_result
    funcs = [lambda x: x + 3, lambda x: x - 1]
    init = 4
    assert aux(funcs, init, identity) == 7, "Test case 4 failed"
    # Explanation:
    # Without handle_result, final result would be 8 (5 -> 7).
    # handle_result doubles it: 16.

    # Test case 5: Complex functions
    funcs = [
        lambda x: x * x,        # Square
        lambda x: x // 2,       # Integer division by 2
        lambda x: x + 10        # Add 10
    ]
    init = 3
    assert aux(funcs, init, identity) == 19, "Test case 5 failed"
    # Explanation:
    # Skip: 3 -> 3 -> 3
    # Apply: 3 -> 9 -> 4 -> 14
    # Max at each step is 49.

    # Test case 6: Nested functions
    funcs = [
        lambda x: x + 1,
        lambda x: x * 2,
        lambda x: x - 1
    ]
    init = 0
    assert aux(funcs, init, identity) == 2, "Test case 6 failed"
    # Explanation:
    # Skip: 0 -> 0 -> 0
    # Apply: 0 -> 1 -> 2 -> 1
    # Max at each step is 2.

    # Test case 7: Handle result overrides everything
    funcs = [lambda x: x * 3]
    init = 5
    assert aux(funcs, init, identity) == 15, "Test case 7 failed"
    # Explanation:
    # Result of computation is overridden by handle_result.

    print("All test cases passed!")

# Run the tests
test_aux()


aux called with acc=1, funcs=['add_four', 'multiply_by_two']
Processing head=add_four with acc=1
  aux called with acc=1, funcs=['multiply_by_two']
  Processing head=multiply_by_two with acc=1
    aux called with acc=1, funcs=[]
    skip_continuation called with skip_result=1
    aux called with acc=2, funcs=[]
    apply_continuation called with apply_result=2
  combined_result=2
  skip_continuation called with skip_result=2
  aux called with acc=5, funcs=['multiply_by_two']
  Processing head=multiply_by_two with acc=5
    aux called with acc=5, funcs=[]
    skip_continuation called with skip_result=5
    aux called with acc=10, funcs=[]
    apply_continuation called with apply_result=10
  combined_result=10
  apply_continuation called with apply_result=10
combined_result=10
    No more funcs. Final result=10
    No more funcs. Final result=10
  Returning result=10
    No more funcs. Final result=10
    No more funcs. Final result=10
  Returning result=10
Returning result=10
All test c

In [7]:
class Node:
    def __init__(self, value, left=None, right=None):
        self.value = value
        self.left = left
        self.right = right

    def __repr__(self):
        return f"Node({self.value}, {self.left}, {self.right})"

def identity(x):
    return x

def insert(tree, value):
    def insert_aux(current, value, cont):
        def nodeify_left(cont, value, right):
            def wrapper(new_left):
                return cont(Node(value=value, left=new_left, right=right))
            return wrapper

        def nodeify_right(cont, left, value):
            def wrapper(new_right):
                return cont(Node(value=value, left=left, right=new_right))
            return wrapper

        if current is None:
            # Leaf case
            return cont(Node(value=value))
        elif value < current.value:
            # Go to the left subtree
            return insert_aux(current.left, value, nodeify_left(cont, current.value, current.right))
        elif value > current.value:
            # Go to the right subtree
            return insert_aux(current.right, value, nodeify_right(cont, current.left, current.value))
        else:
            # Value already exists
            return cont(current)

    return insert_aux(tree, value, identity)



In [9]:
# Create a basic binary search tree manually
tree = Node(5, Node(3), Node(7))

# Insert values into the tree
tree = insert(tree, 6)  # Insert 6
tree = insert(tree, 4)  # Insert 4
tree = insert(tree, 2)  # Insert 2
tree = insert(tree, 8)  # Insert 8

# Print the resulting tree
print(tree)


Node(5, Node(3, Node(2, None, None), Node(4, None, None)), Node(7, Node(6, None, None), Node(8, None, None)))
